In [1]:
import settings

import time
import datetime
import numpy as np
import pandas as pd

import plotly.plotly as py
import plotly.graph_objs as go
import plotly.tools as tls
tls.set_credentials_file(username=settings.plotly_name, api_key=settings.plotly_key)
tls.set_config_file(world_readable=True,sharing='public')

from poloniex import Poloniex
polo = Poloniex()

In [2]:
def loadPolo(pair, period, start, end):
    dates = []
    data={'price':[],'volume':[]}
    raw = polo.returnChartData(currencyPair=pair,period=period,start=start,end=end)
    for i in range(len(raw)):
        dates.append( datetime.datetime.fromtimestamp(int(raw[i]['date'])))
        data['price'].append( float(raw[i]['weightedAverage']))
        data['volume'].append( float(raw[i]['volume'] ))
    return pd.DataFrame( data, index=dates )

In [3]:
def rollingMean(data,length):
    dates = []
    means = []
    for i in range( length, len(data) ):
        total = 0
        totVol = 0
        for j in range(length):
            date = data.index[i-j]
            total += data.price[date] * data.volume[date]
            totVol +=  data.volume[date]
        means.append( total / totVol )
        dates.append( data.index[i] )
    return pd.Series(means,index=dates)

In [4]:
def macd(data,length1,length2):
    data['mean1'] = rollingMean(data,length1)
    data['mean2'] = rollingMean(data,length2)
    data['macd'] = data['mean1'] - data['mean2']
    data['sig1'] = data['macd'] / data['mean2']
    data['sig2'] = data['sig1'] - data['sig1'].rolling(window=length1,center=False).mean()
    return data

In [5]:
def macdTest(data):
    cash = 1.0
    cashes = []
    dates = []
    paid = False
    for i in range(len(data)):
        signal = data.iloc[i]['sig1']
        if not signal: continue
        price = data.iloc[i]['price']
        if not paid and 0 < signal: paid = price
        if paid and 0 > signal:
            cash *= 1.0 * price / paid
            paid = False
        cashes.append(cash)
        dates.append(data.index[i])
    data['cash']=pd.Series(cashes,index=dates)
    return data

In [6]:
def runMacdTest(pair,period,start,end,length1,length2):
    return macdTest( macd( loadPolo(pair,period,start,end),length1,length2))

In [7]:
def macdTestGraph(data, filename):
    fig = tls.make_subplots(rows=5, cols=1, shared_xaxes=True)
    hover_text = data.apply(lambda r: '<br>'.join(['{}: {}'.format(c, r[c]) for c in data.columns]), axis=1)
    fig.append_trace({'x': data.index, 'y': data['price'], 'type': 'scatter', 'name': 'price', 'text': hover_text}, 1, 1)
    fig.append_trace({'x': data.index, 'y': data['mean1'], 'type': 'scatter', 'name': 'mean1', 'text': hover_text}, 1, 1)
    fig.append_trace({'x': data.index, 'y': data['mean2'], 'type': 'scatter', 'name': 'mean2', 'text': hover_text}, 1, 1)
    fig.append_trace({'x': data.index, 'y': data['cash'], 'type': 'scatter', 'name': 'cash', 'text': hover_text}, 2, 1)
    fig.append_trace({'x': data.index, 'y': data['sig1'], 'type': 'bar', 'name': 'sig1', 'text': hover_text}, 3, 1)
    fig.append_trace({'x': data.index, 'y': data['sig2'], 'type': 'bar', 'name': 'sig2', 'text': hover_text}, 4, 1)
    fig.append_trace({'x': data.index, 'y': data['volume'], 'type': 'bar', 'name': 'volume', 'text': hover_text}, 5, 1)
    return py.iplot(fig, filename=filename)

In [ ]:
def macdHeat(pair,period,start,end,length1,length2, delta):
    data = loadPolo(pair,period,start,end)
    heat = np.full((length2 - length1,length2-length1),1.0)
    for i in range(length1, length2 - delta):
        for j in range( i + delta, length2):
            point = macdTest( macd( data, i, j) )
            heat[i - length1][j - length1] = point.iloc[len(point) - 1]['cash']
    return heat

In [ ]:
heat = macdHeat('USDT_BTC',86400,time.mktime(datetime.date(2015,5,1).timetuple()),time.time(),5,50,2)
py.iplot( [ go.Heatmap(z=heat,x=[5,6],y=[7,6]) ] , filename='heat_usdt_btc')

In [ ]:
data = runMacdTest('USDT_BTC',86400,time.mktime(datetime.date(2015,5,1).timetuple()),time.time(),11,21)
macdTestGraph(data,'max_usdt_btc')

In [ ]:
heat = macdHeat('BTC_ETH',86400,time.mktime(datetime.date(2015,8,10).timetuple()),time.time(),5,50,2)
py.iplot( [ go.Heatmap(z=heat,x=[5,6],y=[7,8])] , filename='heat_btc_eth')

In [ ]:
data = runMacdTest('BTC_ETH',86400,time.mktime(datetime.date(2015,5,1).timetuple()),time.time(),20,31)
macdTestGraph(data,'max_btc_eth')

In [ ]:
heat = macdHeat('USDT_ETH',86400,time.mktime(datetime.date(2015,8,10).timetuple()),time.time(),5,50,2)
py.iplot( [ go.Heatmap(z=heat,x=[5,6],y=[7,8])] , filename='heat_usdt_eth')

In [ ]:
data = runMacdTest('USDT_ETH',86400,time.mktime(datetime.date(2015,8,10).timetuple()),time.time(),15,32)
macdTestGraph(data,'max_btc_eth')

In [ ]:
heat = macdHeat('BTC_NMC',86400,time.mktime(datetime.date(2015,8,10).timetuple()),time.time(),5,50,2)
py.iplot( [ go.Heatmap(z=heat,x=[5,6],y=[7,8])] , filename='heat_btc_nmc')